### **Google Drive Bağlantısı**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### **YOLOX için Kurulumlar**

In [ ]:
!git clone https://github.com/yasarniyazoglu/YOLOX-Custom-Dataset.git
%mv YOLOX-Custom-Dataset YOLOX
%cd YOLOX
!pip3 install -U pip && pip3 install -r requirements.txt
!pip3 install -v -e .  
!pip uninstall -y torch torchvision torchaudio
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

### **Nvidia Apex Kurulumu**

In [ ]:
%cd /content/
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

### **PyCocoTools Kurulumu**

In [ ]:
!pip3 install cython; pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

### **Zipten Çıkarma**

In [ ]:
%cd /content/
!unzip "/content/drive/MyDrive/yoloxxx.zip"

### **Verilerimizi YOLOX'e Uygun Şekilde Biçimlendirme**

In [ ]:
%cd /content/YOLOX/
!ln -s /content/yoloxxx/train ./datasets/VOCdevkit

In [ ]:
%mkdir "/content/YOLOX/datasets/VOCdevkit/VOC2007"
!python3 voc_txt.py "/content/YOLOX/datasets/VOCdevkit/"
%mkdir "/content/YOLOX/datasets/VOCdevkit/VOC2012"
!cp -r "/content/YOLOX/datasets/VOCdevkit/VOC2007/." "/content/YOLOX/datasets/VOCdevkit/VOC2012"

### **Sınıfları Değiştirme**


In [ ]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def sınıf_yazici(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%sınıf_yazici /content/YOLOX/yolox/data/datasets/voc_classes.py

VOC_CLASSES =  (
  'raspberry',
  'multimetre',
  'salca'
)

In [ ]:
%%sınıf_yazici /content/YOLOX/yolox/data/datasets/coco_classes.py

COCO_CLASSES = (
  'raspberry',
  'multimetre',
  'salca'
)

### **Sınıf Sayısını Değiştirme**

In [ ]:
NUM_CLASSES = 3
!sed -i -e 's/self.num_classes = 20/self.num_classes = {NUM_CLASSES}/g' "/content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py"

### **Önceden Eğitilmiş Ağırlıkları İndirme**

In [ ]:
%cd /content/
!wget https://github.com/Megvii-BaseDetection/storage/releases/download/0.0.1/yolox_x.pth
%cd /content/YOLOX/

### **Model Eğitimi**

In [ ]:
!python tools/train.py -f exps/example/yolox_voc/yolox_voc_s.py -d 1 -b 8 --fp16 -o -c /content/yolox_x.pth

### **Model Testi**

In [ ]:
MODEL = "/content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth.tar"
!python3 tools/eval.py -n  yolox-s -c {MODEL} -b 64 -d 1 --conf 0.001 -f exps/example/yolox_voc/yolox_voc_s.py

## **Resim Üzerinden Test**


In [ ]:
MODEL = "/content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth.tar"
FOTO = "/content/YOLOX/assets/IMG_20210510_085842926.jpg"
!python tools/demo.py image -f /content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py -c {MODEL} --path {FOTO} --conf 0.60  --tsize 640 --save_result --device gpu

## **Video Üzerinden Test**


In [ ]:
MODEL = "/content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth.tar"
DATA = "/content/video_salca.mp4"
!python tools/demo.py video -f /content/YOLOX/exps/example/yolox_voc/yolox_voc_s.py -c {MODEL} --path {DATA} --conf 0.70  --tsize 640 --save_result --device gpu

### **Ağırlıkları Drive Kaydetme**

In [ ]:
%cp /content/YOLOX/YOLOX_outputs/yolox_voc_s/latest_ckpt.pth.tar /content/drive/MyDrive/custom_yolo_model